In [1]:
#First application of my nn module for training an agent to play cartpole
#url reference: https://pythonprogramming.net/openai-cartpole-neural-network-example-machine-learning-tutorial/


In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import gym
import random
from statistics import median, mean
from collections import Counter

from nnCode import * #my neural network module



In [3]:
#Data loading and preprocessing...
train = pd.read_csv("mnist_train.csv", delimiter= ",")
X = train.iloc[:, 1:]
y = train.iloc[:, 0]

#Loading test set
test = pd.read_csv("mnist_test.csv", delimiter= ",")
x_test = test.iloc[:, 1:]
y_test = test.iloc[:, 0]

# scaler = preprocessing.StandardScaler().fit(X)

# X = scaler.transform(X)
# test_X = scaler.transform( test_X )

scaler = preprocessing.StandardScaler()
X = scaler.fit_transform( X )
x_test = scaler.transform(x_test)


In [4]:
#Training...

#sizes2 = [784,100, 10]
# EPOCHS = 50
# #sizes2 = [4, 5,  3]
# net = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
#             epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.95)
#net = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = MSE(),
            #epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.07)
#net.train(X,y)

#net.SGD(X , np.array(y), x_test, y_test, minibatch_size = 10)
#net.SGD(X[:10000] , y[:10000], x_test, y_test, minibatch_size = 10)

In [5]:
import matplotlib.pyplot as plt
import gym
from IPython import display
%matplotlib inline

env = gym.make('CartPole-v0')
env.reset()

# for i in range(50):
#    plt.imshow(env.render(mode='rgb_array'))
#    display.display(plt.gcf())    
#    display.clear_output(wait=True)
#    env.step(env.action_space.sample()) # take a random action

# env.close()

array([-0.04783664, -0.01632038,  0.01746689,  0.04167991], dtype=float32)

In [6]:
LR = 1e-3
env = gym.make("CartPole-v0")
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

def initial_population():
    # [OBS, MOVES]
    training_data = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,2)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                # saving our training data
                training_data.append([data[0], output])

        # reset env to play again
        env.reset()
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [7]:
training_data = initial_population()


Average accepted score: 62.115492957746476
Median score for accepted scores: 59.0
Counter({50.0: 28, 54.0: 24, 53.0: 23, 52.0: 22, 51.0: 17, 58.0: 17, 60.0: 16, 57.0: 15, 56.0: 15, 59.0: 13, 61.0: 13, 55.0: 13, 63.0: 12, 65.0: 11, 64.0: 10, 62.0: 10, 70.0: 9, 67.0: 7, 72.0: 7, 74.0: 6, 68.0: 5, 73.0: 5, 66.0: 4, 77.0: 4, 78.0: 4, 71.0: 4, 69.0: 4, 81.0: 4, 83.0: 3, 75.0: 3, 82.0: 3, 76.0: 3, 79.0: 2, 99.0: 1, 84.0: 1, 100.0: 1, 94.0: 1, 117.0: 1, 101.0: 1, 88.0: 1, 87.0: 1, 105.0: 1, 96.0: 1, 125.0: 1, 80.0: 1, 86.0: 1, 102.0: 1, 95.0: 1, 85.0: 1, 91.0: 1, 107.0: 1, 110.0: 1})


C:\Users\malfaro\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
EPOCHS = 50
sizes_game = [4,128,2]
net = Dense(sizes_game, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = LR)

In [9]:
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])
    

In [10]:
X.shape

(21696, 4)

In [11]:
y[:2]

array([[1, 0],
       [1, 0]])

In [12]:
net.forward(X[:2])
net.activated_layers[-1]

array([[0.96217832, 0.03782168],
       [0.97032277, 0.02967723]])

In [13]:
net.metric.get_accuracy(net.activated_layers[-1],y[:2] )

1.0

In [14]:
np.argmax([[0,1], [1,0]], axis = 1)

array([1, 0], dtype=int64)

In [15]:
np.array(training_data)[0]

C:\Users\malfaro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


array([array([ 0.00032196, -0.21832314,  0.01673224,  0.30081907], dtype=float32),
       list([1, 0])], dtype=object)

In [16]:
#sklearn split
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
scaler = preprocessing.StandardScaler()
X_train= scaler.fit_transform( X_train )
X_test = scaler.transform(X_test)

In [24]:
sizes_game = [4,128,2]
LR = 0.01
EPOCHS = 5000
net = Dense(sizes_game, activations = [Sigmoid(), Softmax()],
            Loss = CategoricalCrossEntropyLoss(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = LR)

In [25]:
net.SGD(X_train , y_train, X_test, y_test, minibatch_size = 10)

SGD Training......
Average Error epoch 10: 0.6992073548273233---Average Accuracy: 0.5831336405529954
Accuracy in test set: 0.5810287610619469
Average Error epoch 20: 0.6870170040104899---Average Accuracy: 0.5904147465437787
Accuracy in test set: 0.5832411504424779
Average Error epoch 30: 0.6805851903792218---Average Accuracy: 0.5906912442396314
Accuracy in test set: 0.5860066371681416
Average Error epoch 40: 0.676220324258813---Average Accuracy: 0.5926267281105991
Accuracy in test set: 0.5902470501474927
Average Error epoch 50: 0.673297995094828---Average Accuracy: 0.5930875576036866
Accuracy in test set: 0.5918141592920354
Average Error epoch 60: 0.6712139427992078---Average Accuracy: 0.5943778801843318
Accuracy in test set: 0.5932890855457227
Average Error epoch 70: 0.6694915798800585---Average Accuracy: 0.5946543778801843
Accuracy in test set: 0.59577802359882
Average Error epoch 80: 0.6681157278957522---Average Accuracy: 0.5975115207373272
Accuracy in test set: 0.5972529498525073
A

Average Error epoch 680: 0.6599884337752225---Average Accuracy: 0.6071889400921658
Accuracy in test set: 0.6011246312684366
Average Error epoch 690: 0.6601061023453583---Average Accuracy: 0.6063594470046083
Accuracy in test set: 0.6017699115044248
Average Error epoch 700: 0.660195795679396---Average Accuracy: 0.6059907834101382
Accuracy in test set: 0.6026917404129793
Average Error epoch 710: 0.6602580875290384---Average Accuracy: 0.6061751152073733
Accuracy in test set: 0.6020464601769911
Average Error epoch 720: 0.6602964409016887---Average Accuracy: 0.6053456221198156
Accuracy in test set: 0.6011246312684366
Average Error epoch 730: 0.6602925054899171---Average Accuracy: 0.6057142857142858
Accuracy in test set: 0.6008480825958702
Average Error epoch 740: 0.660220843221953---Average Accuracy: 0.6048847926267281
Accuracy in test set: 0.5999262536873157
Average Error epoch 750: 0.6601174280818296---Average Accuracy: 0.604516129032258
Accuracy in test set: 0.6006637168141593
Average Err

Average Error epoch 1340: 0.6616819182043947---Average Accuracy: 0.606820276497696
Accuracy in test set: 0.6031526548672567
Average Error epoch 1350: 0.6614968858747932---Average Accuracy: 0.6058986175115207
Accuracy in test set: 0.6031526548672567
Average Error epoch 1360: 0.6613041039114597---Average Accuracy: 0.6055299539170507
Accuracy in test set: 0.6020464601769911
Average Error epoch 1370: 0.6611092289981112---Average Accuracy: 0.6058064516129031
Accuracy in test set: 0.6016777286135693
Average Error epoch 1380: 0.6609567316200231---Average Accuracy: 0.6045161290322582
Accuracy in test set: 0.6017699115044248
Average Error epoch 1390: 0.6609079033729339---Average Accuracy: 0.604516129032258
Accuracy in test set: 0.601401179941003
Average Error epoch 1400: 0.6609108690462201---Average Accuracy: 0.6051612903225806
Accuracy in test set: 0.6004793510324484
Average Error epoch 1410: 0.6609523015137116---Average Accuracy: 0.6047004608294931
Accuracy in test set: 0.600202802359882
Aver

Average Error epoch 2000: 0.6669623992987598---Average Accuracy: 0.6014746543778801
Accuracy in test set: 0.5944874631268436
Average Error epoch 2010: 0.6672924773375493---Average Accuracy: 0.6013824884792627
Accuracy in test set: 0.5952249262536873
Average Error epoch 2020: 0.6676086704675105---Average Accuracy: 0.6017511520737328
Accuracy in test set: 0.5961467551622419
Average Error epoch 2030: 0.6678435784353647---Average Accuracy: 0.6010138248847925
Accuracy in test set: 0.5952249262536873
Average Error epoch 2040: 0.6680657379804767---Average Accuracy: 0.5988018433179724
Accuracy in test set: 0.5966998525073747
Average Error epoch 2050: 0.6682850346394867---Average Accuracy: 0.5988940092165899
Accuracy in test set: 0.5975294985250738
Average Error epoch 2060: 0.6684732808710804---Average Accuracy: 0.6002764976958526
Accuracy in test set: 0.5963311209439528
Average Error epoch 2070: 0.6686799993360217---Average Accuracy: 0.6010138248847925
Accuracy in test set: 0.5959623893805309


Average Error epoch 2660: 0.6629541081127898---Average Accuracy: 0.6033179723502303
Accuracy in test set: 0.5983591445427728
Average Error epoch 2670: 0.6627345681971519---Average Accuracy: 0.6028571428571429
Accuracy in test set: 0.5973451327433629
Average Error epoch 2680: 0.6625404334399041---Average Accuracy: 0.6024884792626729
Accuracy in test set: 0.5967920353982301
Average Error epoch 2690: 0.6623616877423771---Average Accuracy: 0.6025806451612903
Accuracy in test set: 0.5961467551622419
Average Error epoch 2700: 0.6621736587568692---Average Accuracy: 0.6023963133640552
Accuracy in test set: 0.5962389380530974
Average Error epoch 2710: 0.6620015542545791---Average Accuracy: 0.6023963133640552
Accuracy in test set: 0.5962389380530974
Average Error epoch 2720: 0.6618444580761313---Average Accuracy: 0.6021198156682027
Accuracy in test set: 0.5968842182890856
Average Error epoch 2730: 0.6617088711547843---Average Accuracy: 0.6026728110599078
Accuracy in test set: 0.5972529498525073


Average Error epoch 3320: 0.6615259965279839---Average Accuracy: 0.6008294930875576
Accuracy in test set: 0.5944874631268436
Average Error epoch 3330: 0.6617304516512752---Average Accuracy: 0.6009216589861751
Accuracy in test set: 0.59375
Average Error epoch 3340: 0.6619291920750953---Average Accuracy: 0.60073732718894
Accuracy in test set: 0.5943952802359882
Average Error epoch 3350: 0.6621327930550662---Average Accuracy: 0.6013824884792627
Accuracy in test set: 0.594948377581121
Average Error epoch 3360: 0.6623390234956504---Average Accuracy: 0.6002764976958525
Accuracy in test set: 0.5940265486725663
Average Error epoch 3370: 0.6625156416391407---Average Accuracy: 0.6001843317972351
Accuracy in test set: 0.5943030973451328
Average Error epoch 3380: 0.6626547717019275---Average Accuracy: 0.6002764976958525
Accuracy in test set: 0.5939343657817109
Average Error epoch 3390: 0.6627723413627826---Average Accuracy: 0.5999078341013824
Accuracy in test set: 0.5936578171091446
Average Error 

Average Error epoch 3990: 0.6631963967507679---Average Accuracy: 0.5991705069124423
Accuracy in test set: 0.5928281710914455
Average Error epoch 4000: 0.6632640449778913---Average Accuracy: 0.5991705069124423
Accuracy in test set: 0.5930125368731564
Average Error epoch 4010: 0.6633329611788493---Average Accuracy: 0.5998156682027649
Accuracy in test set: 0.5924594395280236
Average Error epoch 4020: 0.6634019224815858---Average Accuracy: 0.5994470046082948
Accuracy in test set: 0.5924594395280236
Average Error epoch 4030: 0.6634665724782277---Average Accuracy: 0.5989861751152072
Accuracy in test set: 0.5919063421828908
Average Error epoch 4040: 0.663524751523822---Average Accuracy: 0.5993548387096774
Accuracy in test set: 0.5914454277286135
Average Error epoch 4050: 0.663579148867923---Average Accuracy: 0.600184331797235
Accuracy in test set: 0.5915376106194691
Average Error epoch 4060: 0.6636261623809598---Average Accuracy: 0.60036866359447
Accuracy in test set: 0.5914454277286135
Avera

Average Error epoch 4650: 0.6621372207180488---Average Accuracy: 0.6030414746543779
Accuracy in test set: 0.5943952802359882
Average Error epoch 4660: 0.6621557513603498---Average Accuracy: 0.6031336405529953
Accuracy in test set: 0.5945796460176991
Average Error epoch 4670: 0.6621770811806825---Average Accuracy: 0.6035023041474654
Accuracy in test set: 0.5944874631268436
Average Error epoch 4680: 0.662206361426546---Average Accuracy: 0.6035944700460829
Accuracy in test set: 0.5946718289085545
Average Error epoch 4690: 0.6622347080942934---Average Accuracy: 0.6027649769585254
Accuracy in test set: 0.5950405604719764
Average Error epoch 4700: 0.6622627665728725---Average Accuracy: 0.6029493087557604
Accuracy in test set: 0.5948561946902655
Average Error epoch 4710: 0.6622832553201273---Average Accuracy: 0.6029493087557604
Accuracy in test set: 0.5946718289085545
Average Error epoch 4720: 0.6622998397487246---Average Accuracy: 0.6033179723502304
Accuracy in test set: 0.5947640117994101
A

In [ ]:
X_train.shape

In [ ]:
X_train[:2]

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
clf = MLPClassifier(random_state=1, max_iter=3000).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
X, y = make_classification(n_samples=30000, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
sizes_game = [20,128,2]
LR = 0.01
EPOCHS = 50
net = Dense(sizes_game, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = LR)

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
net.SGD(X_train , y_train, X_test, y_test, minibatch_size = 10)